In [337]:
#FIRST HALF OF THIS CODE COPIED FROM LESSON 1
# these allow you to modify source code of imports, notebook automatically updates
%load_ext autoreload
%autoreload 2
# show plots in notebook
%matplotlib inline 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [338]:
from fastai.fastai.imports import *
from fastai.old.fastai.structured import *

# from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [339]:
PATH1 = 'data/BAYZF.csv'
PATH2 = 'data/HON.csv'
PATH3 = 'data/MMM.csv'
PATH4 = 'data/SYF.csv'
dateColumn = "Date"
df_BAYZF = pd.read_csv(PATH1, low_memory=False, parse_dates=[dateColumn])
df_HON = pd.read_csv(PATH2, low_memory=False, parse_dates=[dateColumn])
df_MMM = pd.read_csv(PATH3, low_memory=False, parse_dates=[dateColumn])
df_SYF = pd.read_csv(PATH4, low_memory=False, parse_dates=[dateColumn])

In [340]:
df_BAYZF.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-02-13,55.549999,55.549999,55.549999,55.549999,44.911427,100.0
1,2009-02-17,53.400002,53.400002,53.099998,53.200001,43.011486,400.0
2,2009-02-18,53.400002,53.400002,53.299999,53.299999,43.092339,300.0
3,2009-02-19,54.250000,54.250000,53.750000,53.750000,43.456146,500.0
4,2009-02-20,52.299999,52.299999,51.900002,51.910000,41.968533,500.0


In [341]:
df_raw = df_BAYZF

In [342]:
add_datepart(df_raw, dateColumn)

In [343]:
def add_next(df, column_name):
    # makes a new column called "next" + column_name that includes the next value in that column for prediction purposes
    # ex: add a new column called next_open to predict the next opening price
    # removes the last row from the whole dataframe as it is null for the prediction
    df_to_add = pd.DataFrame(df[column_name])
    df_to_add.drop([0], axis=0, inplace=True)
    df.drop([len(df)-1], axis=0, inplace=True)
    df['next_'+column_name] = df_to_add.values

add_next(df_raw, 'Open')
print(df_raw.head())

        Open       High        Low      Close  Adj Close  Volume    Year  \
0  55.549999  55.549999  55.549999  55.549999  44.911427   100.0  2009.0   
1  53.400002  53.400002  53.099998  53.200001  43.011486   400.0  2009.0   
2  53.400002  53.400002  53.299999  53.299999  43.092339   300.0  2009.0   
3  54.250000  54.250000  53.750000  53.750000  43.456146   500.0  2009.0   
4  52.299999  52.299999  51.900002  51.910000  41.968533   500.0  2009.0   

   Month  Week   Day  Dayofweek  Dayofyear  Is_month_end  Is_month_start  \
0    2.0   7.0  13.0        4.0       44.0         False           False   
1    2.0   8.0  17.0        1.0       48.0         False           False   
2    2.0   8.0  18.0        2.0       49.0         False           False   
3    2.0   8.0  19.0        3.0       50.0         False           False   
4    2.0   8.0  20.0        4.0       51.0         False           False   

   Is_quarter_end  Is_quarter_start  Is_year_end  Is_year_start     Elapsed  \
0      

In [344]:
# goes through each column, find the nulls, adds em, divides by the length
# kind of op syntax here
# df_raw.isnull returns a new dataframe with true for null and false for non-null from original df
df_raw.isnull().sum().sort_index()/len(df_raw)

Adj Close           0.009558
Close               0.009558
Day                 0.009558
Dayofweek           0.009558
Dayofyear           0.009558
Elapsed             0.000000
High                0.009558
Is_month_end        0.000000
Is_month_start      0.000000
Is_quarter_end      0.000000
Is_quarter_start    0.000000
Is_year_end         0.000000
Is_year_start       0.000000
Low                 0.009558
Month               0.009558
Open                0.009558
Volume              0.009558
Week                0.009558
Year                0.009558
next_Open           0.009956
dtype: float64

In [345]:
to_predict = "next_Open"
# options: ["Adj Close", "Open", "High", "Low", "Volume"]
to_drop = ["Adj Close", "Open", "High", "Low", "Volume"]
result = proc_df(df_raw, y_fld=to_predict, skip_flds=to_drop)
df = result[0]
y = result[1]

In [346]:
df.columns

Index(['Close', 'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
       'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
       'Is_year_end', 'Is_year_start', 'Elapsed', 'Close_na', 'Year_na',
       'Month_na', 'Week_na', 'Day_na', 'Dayofweek_na', 'Dayofyear_na'],
      dtype='object')

In [347]:
y

array([53.4 , 53.4 , 54.25, ...,   nan,   nan,   nan])

In [348]:
to_remove = np.argwhere(np.isnan(y))
to_remove = [row[0] for row in to_remove.tolist()]
print(to_remove)
df = df.drop(to_remove, axis=0)
y = y[~np.isnan(y)]

[2486, 2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510]


In [349]:
# splits dataset based on how many you want to train/validate
def split_vals(a, n):
    return a[:n].copy(), a[n:].copy()

n_valid = 42 # for the final year
n_train = len(df) - n_valid
# raw just uses the df_raw
raw_train, raw_valid = split_vals(df_raw, n_train)
X_train_close, X_valid_close = split_vals(df, n_train)
y_train_close, y_valid_close = split_vals(y, n_train)

In [350]:
def rmse(x, y):
    return math.sqrt(((x-y)**2).mean())

# function for print r2 and RMSE of training data and validation data
def print_score(rf):
    res = [rmse(rf.predict(X_train), y_train), rmse(rf.predict(X_valid), y_valid), rf.score(X_train, y_train), rf.score(X_valid, y_valid)]
    print(res)

In [351]:
rf_next_open = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
rf_next_open.fit(X_train_close, y_train_close)
print_score(rf_next_open)

[0.9696306633087193, 2.7079003807127133, 0.9989847746183244, 0.9815601448603521]


In [352]:
len(X_train_close.columns)

21

In [353]:
# now making a model to PREDICT CLOSE USING OPEN
# don't need add_next as close happens after open in a day
df_BAYZF = pd.read_csv(PATH1, low_memory=False, parse_dates=[dateColumn])
df_raw = df_BAYZF
add_datepart(df_raw, dateColumn)
to_predict = "Close"
# options: ["Adj Close", "Open", "High", "Low", "Volume"]
to_drop = ["Adj Close", "High", "Low", "Volume"]
result = proc_df(df_raw, y_fld=to_predict, skip_flds=to_drop)
df = result[0]
y = result[1]

to_remove = np.argwhere(np.isnan(y))
to_remove = [row[0] for row in to_remove.tolist()]
print(to_remove)
df = df.drop(to_remove, axis=0)
y = y[~np.isnan(y)]

n_valid = 42 # for the final year
n_train = len(df) - n_valid
# raw just uses the df_raw
raw_train, raw_valid = split_vals(df_raw, n_train)
X_train, X_valid = split_vals(df, n_train)
y_train, y_valid = split_vals(y, n_train)

rf_next_close = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
rf_next_close.fit(X_train, y_train)
print_score(rf_next_close)

[2487, 2488, 2489, 2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 2511]
[0.5343255285177207, 9.790349071081538, 0.9996735684605043, -4.008817988001707]


In [354]:
raw_train

,Open,High,Low,Close,Adj Close,Volume,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,55.549999,55.549999,55.549999,55.549999,44.911427,100.0,2009.0,2.0,7.0,13.0,4.0,44.0,False,False,False,False,False,False,1234483200
1,53.400002,53.400002,53.099998,53.200001,43.011486,400.0,2009.0,2.0,8.0,17.0,1.0,48.0,False,False,False,False,False,False,1234828800
2,53.400002,53.400002,53.299999,53.299999,43.092339,300.0,2009.0,2.0,8.0,18.0,2.0,49.0,False,False,False,False,False,False,1234915200
3,54.250000,54.250000,53.750000,53.750000,43.456146,500.0,2009.0,2.0,8.0,19.0,3.0,50.0,False,False,False,False,False,False,1235001600
4,52.299999,52.299999,51.900002,51.910000,41.968533,500.0,2009.0,2.0,8.0,20.0,4.0,51.0,False,False,False,False,False,False,1235088000
5,53.540001,53.540001,51.750000,52.000000,42.041302,1100.0,2009.0,2.0,9.0,23.0,0.0,54.0,False,False,False,False,False,False,1235347200
6,52.500000,52.500000,51.250000,51.250000,41.434929,2500.0,2009.0,2.0,9.0,24.0,1.0,55.0,False,False,False,False,False,False,1235433600
7,50.660000,50.660000,50.660000,50.660000,40.957924,100.0,2009.0,2.0,9.0,25.0,2.0,56.0,False,False,False,False,False,False,1235520000
8,48.500000,48.500000,48.500000,48.500000,39.211594,100.0,2009.0,2.0,9.0,26.0,3.0,57.0,False,False,False,False,False,False,1235606400
9,47.700001,48.650002,47.700001,48.500000,39.211594,1700.0,2009.0,2.0,9.0,27.0,4.0,58.0,False,False,False,False,False,False,1235692800


In [360]:
y_valid

array([78.2 , 77.7 , 81.4 , 79.3 , 79.42, 82.45, 84.65, 80.68, 80.68, 77.8 , 76.33, 74.9 , 72.04, 71.33,
       70.9 , 69.16, 71.07, 70.42, 73.41, 71.75, 73.  , 73.  , 73.56, 74.94, 72.12, 70.36, 71.06, 69.76,
       70.63, 75.4 , 71.99, 71.36, 72.11, 72.21, 69.  , 68.97, 68.28, 68.96, 68.3 , 67.46, 68.87, 69.5 ])

In [365]:
# so now the model is shown to predict the next open given close with R2 of 0.803 as seen on validate set
# this data could be combined for more robust data
# now we will try to chain the last 42 days by using close on the last 43rd day to predict open on 365, then
# using that to predict close on that day, then using that to predict next_open, then chain all the way for the year

def make_df_from_seed(seed, count, predOpen):
    if (predOpen):
        newDf = X_valid.iloc[count].to_frame().transpose()
        newDf["Open"] = seed
    else:
        newDf = X_valid_close.iloc[count].to_frame().transpose()
        newDf["Close"] = seed
    return newDf

df_BAYZF = pd.read_csv(PATH1, low_memory=False, parse_dates=[dateColumn])
df_raw = df_BAYZF
add_datepart(df_raw, dateColumn)
final_close_seed = X_train_close.drop([row for row in range(len(X_train_close)-1)], axis=0)
print(final_close_seed)

def pred_next_open(seed):
    return rf_next_open.predict(seed)

def pred_next_close(seed):
    return rf_next_close.predict(seed)

def chain_preds(seed_df):
    yearLongOpenPreds = []
    yearLongClosePreds = []
    count = 0
    while (count != 42):
        next_open = pred_next_open(seed_df)
        seed = next_open
        seed_df = make_df_from_seed(seed, count, True)
        
        next_close = pred_next_close(seed_df)
        seed = next_close
        seed_df = make_df_from_seed(seed, count, False)
        
        yearLongOpenPreds.append(next_open)
        yearLongClosePreds.append(next_close)
        count += 1
        
    return yearLongOpenPreds, yearLongClosePreds

yearLongOpenPreds, yearLongClosePreds = chain_preds(final_close_seed)

      Close    Year  Month  Week   Day  Dayofweek  Dayofyear  Is_month_end  \
2443   75.0  2018.0   10.0  43.0  26.0        4.0      299.0         False   

      Is_month_start  Is_quarter_end      ...       Is_year_end  \
2443           False           False      ...             False   

      Is_year_start     Elapsed  Close_na  Year_na  Month_na  Week_na  Day_na  \
2443          False  1540512000     False    False     False    False   False   

      Dayofweek_na  Dayofyear_na  
2443         False         False  

[1 rows x 21 columns]


In [366]:
rmse(yearLongClosePreds, y_valid_close)

13.202700304671772

In [367]:
mean_result = np.full((len(yearLongClosePreds), 1), y_valid_close.mean())
rmse(yearLongClosePreds, mean_result)

12.56676697409441

In [368]:
yearLongClosePreds

[array([78.78438]),
 array([79.38076]),
 array([82.22129]),
 array([82.28499]),
 array([82.46608]),
 array([83.72367]),
 array([83.58644]),
 array([83.45602]),
 array([83.42978]),
 array([83.988]),
 array([83.88053]),
 array([83.8303]),
 array([83.75611]),
 array([83.78656]),
 array([83.86459]),
 array([84.87275]),
 array([86.90665]),
 array([86.90551]),
 array([87.15722]),
 array([87.04975]),
 array([86.61711]),
 array([86.46422]),
 array([86.90876]),
 array([88.15899]),
 array([88.39446]),
 array([87.25297]),
 array([86.9962]),
 array([87.68247]),
 array([88.20419]),
 array([88.12592]),
 array([88.01686]),
 array([87.88846]),
 array([88.49562]),
 array([88.38816]),
 array([88.76448]),
 array([88.50078]),
 array([88.37238]),
 array([88.66265]),
 array([88.56549]),
 array([87.57735]),
 array([88.02286]),
 array([90.32787])]

In [369]:
y_valid_close

array([76.25, 79.  , 78.76, 79.3 , 80.12, 81.03, 84.65, 80.68, 80.68, 80.  , 74.43, 74.47, 72.07, 72.07,
       71.29, 69.76, 71.41, 70.42, 72.2 , 73.35, 72.43, 72.01, 74.83, 74.68, 74.51, 72.35, 71.63, 69.43,
       71.57, 74.93, 72.05, 72.  , 73.8 , 72.21, 71.53, 70.25, 69.21, 68.3 , 68.39, 69.  , 68.98, 69.89])